# Poseidon Setup Between Chameleon And Fabric

In [ ]:
import sys
import os
import json
import traceback
import time
from datetime import datetime, timedelta
from dateutil import tz
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

## Chameleon Imports

In [ ]:
import chi
import chi.lease

## Fabric Imports

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

In [ ]:
try:
    fablib.list_sites(
        force_refresh=True, 
        fields=[
            "name", 
            "cores_available", 
            "ram_available",
            "disk_available",
            "nic_basic_available", 
            "nic_connectx_5_available", 
            "nic_connectx_6_available",
            "nvme_available",
            "tesla_t4_available",
            "rtx6000_available",
            "a30_available",
            "a40_available",
            "fpga_u280_available"
        ]
    )
except Exception as e:
    print(f"Exception: {e}")

## Chameleon Variables

For stitching to work we need to specify stich provider "fabric" and physical network "physnet1"

In [ ]:
# Chameleon Config
chameleon_prefix =  f"poseidon-"
chameleon_server_name = chameleon_prefix+'worker'
chameleon_network_name = chameleon_prefix+'net'
chameleon_subnet_name = chameleon_prefix+'subnet'
chameleon_router_name = chameleon_prefix+'router'
chameleon_lease_name = chameleon_prefix+'lease'

chameleon_image_name='CC-Ubuntu20.04'
chameleon_node_type="compute_cascadelake_r"
chameleon_physical_network='physnet1'
chameleon_stitch_provider='fabric'
chameleon_server_count=1
chameleon_key_name='fabric-chameleon'

#Chameleon Network Config
chameleon_subnet_addr = "192.168.100.0/24"
chameleon_allocation_pool_start='192.168.100.10'
chameleon_allocation_pool_end='192.168.100.20'
chameleon_gateway_ip_addr='192.168.100.1'

## Fabric Variables

In [ ]:
# Create a FABlib manager
fablib.show_config()

# FABRIC Config
fabric_prefix =  f"poseidon-"
fabric_slice_name = fabric_prefix+'experiment'
fabric_os_image='default_ubuntu_20'

fabric_facility_port_name = "Chameleon-StarLight"
fabric_facility_port_site = "STAR"

fabric_submit_name = fabric_prefix+'submit'
fabric_submit_site = 'STAR'
fabric_submit_host = "star-w3.fabric-testbed.net"
fabric_submit_cores = 16
fabric_submit_ram = 32
fabric_submit_disk = 100
fabric_submit_ip_addr ="192.168.101.100"

fabric_data_name = fabric_prefix+'data'
fabric_data_site='DALL'
fabric_data_host="dall-w2.fabric-testbed.net"
fabric_data_cores = 16
fabric_data_ram = 32
fabric_data_disk = 250
fabric_data_ip_addr ="192.168.101.200"

fabric_router_name = fabric_prefix+'router'
fabric_router_site = 'STAR'
fabric_router_cores = 16
fabric_router_ram = 32
fabric_router_disk = 40
fabric_router_ip_addr ="192.168.101.250"
fabric_router_stitch_ip_addr ="192.168.100.250"

#Fabric Network Config
fabric_subnet_addr = "192.168.101.0/24"
fabric_chameleon_network_name = "fabric_chameleon_net"
fabric_network_name = "fabric_net"
#fabric_vlan_value = 1000

## Create Chameleon Lease For Compute and Network

In [ ]:
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

try:
    # Set start/end date for lease
    # Start one minute into future to avoid Blazar thinking lease is in past
    # due to rounding to closest minute.
    start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
    end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

    # Build list of reservations (in this case there is only one reservation)
    reservation_list = []
    chi.lease.add_node_reservation(reservation_list, count=chameleon_server_count, node_type=chameleon_node_type)

    reservation_list.append(
            {
                "resource_type": "network",
                "network_name": chameleon_network_name,
                "network_properties": "",
                "resource_properties": json.dumps(
                    ["==", "$stitch_provider", chameleon_stitch_provider]
                ),
            }
    )
    
    #reservation_list.append(
    #        {
    #            "resource_type": "network",
    #            "network_name": chameleon_network_name,
    #            "network_properties": "",
    #            "resource_properties": json.dumps(
    #                ["==", "$segment_id", "3303"]
    #            ),
    #        }
    #)


    # Create the lease
    chameleon_lease = chi.lease.create_lease(chameleon_lease_name,
                                      reservations=reservation_list,
                                      start_date=start_date,
                                      end_date=end_date)

    #Print the lease info
    chameleon_compute_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
    print(f"chameleon_compute_reservation_id: {chameleon_compute_reservation_id}")
    chameleon_network_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
    print(f"chameleon_network_reservation_id: {chameleon_network_reservation_id}")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Wait Until Network Gets Provisioned

In [ ]:
chameleon_network = None
chameleon_network_id = None
chameleon_network_vlan = None

while chameleon_network_vlan == None:
    try:
        #Get the network
        chameleon_network = chi.network.get_network(chameleon_network_name)

        #Get the network ID
        chameleon_network_id = chameleon_network['id']
        print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        chameleon_network_vlan = chameleon_network['provider:segmentation_id']
        print(f'network_vlan: {chameleon_network_vlan}')
    except:
        print(f'Chameleon Network is not ready. Trying again!')
        time.sleep(10)           

## Create Chameleon's Subnet

In [ ]:
try:
    chameleon_subnet = chi.network.create_subnet(chameleon_subnet_name, chameleon_network_id, 
                                             cidr=str(chameleon_subnet_addr),
                                             allocation_pool_start=chameleon_allocation_pool_start,
                                             allocation_pool_end=chameleon_allocation_pool_end,
                                             gateway_ip=chameleon_gateway_ip_addr)
    chameleon_subnet_id = chameleon_subnet["id"]
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

print(json.dumps(chameleon_subnet, indent=2))

## Create Chameleon's Router and Add The Subnet

In [ ]:
try:
    chameleon_router = chi.network.create_router(chameleon_router_name, gw_network_name='public')
    chameleon_router = chi.network.add_subnet_to_router(chameleon_router['id'], chameleon_subnet_id)
    chameleon_router = chi.network.add_route_to_router(chameleon_router['id'], fabric_subnet_addr, fabric_router_stitch_ip_addr)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()
    
print(json.dumps(chameleon_router, indent=2))

## Create Fabric Slice With Facility Port

Facility ports currently only work with the dedicate Smart Nics (ConnectX_5 and ConnectX_6)
Additionally stitching can work from within the same site that has the Facility Port or from VMs in different sites, but the connection type is L2PTP.
Finally each Facility Port has its own permission on Fabric which the project needs to gain access before using

In [ ]:
try:
    #Create Slice
    fabric_slice = fablib.new_slice(name=fabric_slice_name)
    
    # Add facility port
    fabric_facility_port = fabric_slice.add_facility_port(name=fabric_facility_port_name, site=fabric_facility_port_site, vlan=str(chameleon_network_vlan))
    fabric_facility_port_iface = fabric_facility_port.get_interfaces()[0]
    
    # Add poseidon router node
    fabric_router = fabric_slice.add_node(
                        name=fabric_router_name, 
                        site=fabric_router_site,
                        image=fabric_os_image,
                        cores=fabric_router_cores,
                        ram=fabric_router_ram,
                        disk=fabric_router_disk)
    
    fabric_router_nic = fabric_router.add_component(model='NIC_ConnectX_5', name="fabric_router_nic")
    fabric_router_iface_to_chameleon = fabric_router_nic.get_interfaces()[0]
    #fabric_router_iface_to_chameleon.set_vlan(fabric_vlan_value)
    
    fabric_router_iface_to_fabric = fabric_router_nic.get_interfaces()[1]
    #fabric_router_iface_to_fabric.set_vlan(fabric_vlan_value)
    
    # Add poseidon submit node
    fabric_submit = fabric_slice.add_node(
                        name=fabric_submit_name, 
                        site=fabric_submit_site,
                        host=fabric_submit_host,
                        image=fabric_os_image,
                        cores=fabric_submit_cores,
                        ram=fabric_submit_ram,
                        disk=fabric_submit_disk)
    
    fabric_submit_iface = fabric_submit.add_component(model='NIC_Basic', name="fabric_submit_nic").get_interfaces()[0]
    #fabric_submit_iface.set_vlan(fabric_vlan_value)
    
    # Add poseidon router node
    fabric_data = fabric_slice.add_node(
                    name=fabric_data_name, 
                    site=fabric_data_site,
                    host=fabric_data_host,
                    image=fabric_os_image,
                    cores=fabric_data_cores,
                    ram=fabric_data_ram,
                    disk=fabric_data_disk)
    
    fabric_data_iface = fabric_data.add_component(model='NIC_Basic', name="fabric_data_nic").get_interfaces()[0]
    #fabric_data_iface.set_vlan(fabric_vlan_value)
    
    #Create the l2networks
    fabric_chameleon_net = fabric_slice.add_l2network(name=fabric_chameleon_network_name, interfaces=[fabric_router_iface_to_chameleon,fabric_facility_port_iface])
    fabric_net = fabric_slice.add_l2network(name=fabric_network_name, interfaces=[fabric_router_iface_to_fabric,fabric_submit_iface,fabric_data_iface])

    #Submit the Request
    fabric_slice.submit()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Configure Network on Fabric Router

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=fabric_slice_name)
    fabric_router = fabric_slice.get_node(name=fabric_router_name)
    
    fabric_router_iface_to_chameleon = fabric_router.get_interface(network_name=fabric_chameleon_network_name)
    fabric_router_iface_to_fabric = fabric_router.get_interface(network_name=fabric_network_name)

    fabric_router_iface_to_chameleon_os_name = fabric_router_iface_to_chameleon.get_os_interface()
    fabric_router_iface_to_fabric_os_name = fabric_router_iface_to_fabric.get_os_interface()

    stdout, stderr = fabric_router.execute(f'sudo ip link set dev {fabric_router_iface_to_chameleon_os_name} up', quiet=True)
    #stdout, stderr = fabric_router.execute(f'sudo ip link set mtu 8900 dev {fabric_router_iface_to_chameleon_os_name}', quiet=True)
    stdout, stderr = fabric_router.execute(f'sudo ip addr add {fabric_router_stitch_ip_addr}/24 dev {fabric_router_iface_to_chameleon_os_name}', quiet=True)
    stdout, stderr = fabric_router.execute(f'sudo tc qdisc add dev {fabric_router_iface_to_chameleon_os_name} root fq', quiet=True)

    stdout, stderr = fabric_router.execute(f'sudo ip link set dev {fabric_router_iface_to_fabric_os_name} up', quiet=True)
    #stdout, stderr = fabric_router.execute(f'sudo ip link set mtu 8900 dev {fabric_router_iface_to_fabric_os_name}', quiet=True)
    stdout, stderr = fabric_router.execute(f'sudo ip addr add {fabric_router_ip_addr}/24 dev {fabric_router_iface_to_fabric_os_name}', quiet=True)
    stdout, stderr = fabric_router.execute(f'sudo tc qdisc add dev {fabric_router_iface_to_fabric_os_name} root fq', quiet=True)
    
    #add default gateway
    stdout, stderr = fabric_router.execute(f'sudo ip route add default via {chameleon_gateway_ip_addr}', quiet=True)
    
    #enable ip forwarding
    stdout, stderr = fabric_router.execute('sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
    
    stdout, stderr = fabric_router.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = fabric_router.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Configure Network on Submit Node

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=fabric_slice_name)
    fabric_submit = fabric_slice.get_node(name=fabric_submit_name)
    
    fabric_submit_iface = fabric_submit.get_interface(network_name=fabric_network_name)
    fabric_submit_iface_os_name = fabric_submit_iface.get_os_interface()

    stdout, stderr = fabric_submit.execute(f'sudo ip link set dev {fabric_submit_iface_os_name} up', quiet=True)
    #stdout, stderr = fabric_submit.execute(f'sudo ip link set mtu 8900 dev {fabric_submit_iface_os_name}', quiet=True)
    stdout, stderr = fabric_submit.execute(f'sudo ip addr add {fabric_submit_ip_addr}/24 dev {fabric_submit_iface_os_name}', quiet=True)
    stdout, stderr = fabric_submit.execute(f'sudo tc qdisc add dev {fabric_submit_iface_os_name} root fq', quiet=True)
    
    #add default gateway and route to chameleon subnet via router
    #stdout, stderr = fabric_submit.execute(f'sudo ip route add {chameleon_subnet_address} via {fabric_router_ip_addr}', quiet=True)
    stdout, stderr = fabric_submit.execute(f'sudo ip route add default via {fabric_router_ip_addr}', quiet=True)

    stdout, stderr = fabric_submit.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = fabric_submit.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Configure Network on Data Node

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=fabric_slice_name)
    fabric_data = fabric_slice.get_node(name=fabric_data_name)
    
    fabric_data_iface = fabric_data.get_interface(network_name=fabric_network_name)
    fabric_data_iface_os_name = fabric_data_iface.get_os_interface()

    stdout, stderr = fabric_data.execute(f'sudo ip link set dev {fabric_data_iface_os_name} up', quiet=True)
    #stdout, stderr = fabric_data.execute(f'sudo ip link set mtu 8900 dev {fabric_data_iface_os_name}', quiet=True)
    stdout, stderr = fabric_data.execute(f'sudo ip addr add {fabric_data_ip_addr}/24 dev {fabric_data_iface_os_name}', quiet=True)
    stdout, stderr = fabric_data.execute(f'sudo tc qdisc add dev {fabric_data_iface_os_name} root fq', quiet=True)
    
    #add default gateway and route to chameleon subnet via router
    #stdout, stderr = fabric_data.execute(f'sudo ip route add {chameleon_subnet_address} via {fabric_router_ip_addr}', quiet=True)
    stdout, stderr = fabric_data.execute(f'sudo ip route add default via {fabric_router_ip_addr}', quiet=True)

    stdout, stderr = fabric_data.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = fabric_data.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Sanity Check: Ping Chameleon Router From Fabric

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=fabric_slice_name)
    fabric_node = fabric_slice.get_node(name=fabric_router_name) 
    stdout, stderr = fabric_node.execute(f'ping -c 4 {chameleon_gateway_ip_addr}')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Sanity Check: Ping Fabric Nodes From Fabric Router

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=fabric_slice_name)
    fabric_node = fabric_slice.get_node(name=fabric_router_name) 
    stdout, stderr = fabric_node.execute(f'ping -c 4 {fabric_submit_ip_addr}')
    stdout, stderr = fabric_node.execute(f'ping -c 4 {fabric_data_ip_addr}')
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Sanity Check: Ping Chameleon Router From Fabric Nodes

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=fabric_slice_name)
    fabric_node = fabric_slice.get_node(name=fabric_submit_name) 
    stdout, stderr = fabric_node.execute(f'ping -c 4 {chameleon_gateway_ip_addr}')
    
    fabric_node = fabric_slice.get_node(name=fabric_data_name) 
    stdout, stderr = fabric_node.execute(f'ping -c 4 {chameleon_gateway_ip_addr}')
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Create Chameleon Nodes If Router Responds to Ping

In [ ]:
servers=[]

try:
    for i in range(chameleon_server_count):
        server_name=f"{chameleon_server_name}-{i}"
        
        # Create the server
        servers.append(chi.server.create_server(server_name, 
                              reservation_id=chameleon_compute_reservation_id, 
                              network_name=chameleon_network_name, 
                              image_name=chameleon_image_name,
                              key_name=chameleon_key_name))

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

for server in servers:
    chi.server.wait_for_active(server.id)

In [ ]:
# Give 5 more minutes for the Chameleon Nodes to Stabilize
time.sleep(300)

## Get Chameleon Worker IPs

In [ ]:
chameleon_fixed_ips={}

try:
    for i in range(chameleon_server_count):
        chi_server_name=f"{chameleon_server_name}-{i}"
        chi_server_id = chi.server.get_server_id(chi_server_name)
        chi_fixed_ip = chi.server.get_server(chi_server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
        chameleon_fixed_ips[chi_server_name]=chi_fixed_ip
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()
    
for chi_server_name,chi_fixed_ip in chameleon_fixed_ips.items():
    print(f'{chi_server_name}: {chi_fixed_ip}')

## Sanity Check: Ping Chameleon Nodes From Fabric

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=fabric_slice_name)
    fabric_node = fabric_slice.get_node(name=fabric_submit_name)
    
    for chi_server_name,chi_fixed_ip in chameleon_fixed_ips.items():
        stdout, stderr = fabric_node.execute(f'ping -c 10 {chi_fixed_ip}')
        
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Cleanup Chameleon

In [ ]:
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}-{i}"
    chi.server.delete_server(chi.server.get_server_id(server_name))

# Wait for the nodes to be deleted
time.sleep(200)

In [ ]:
router_id = chameleon_router['router']['id']
subnet_id = chameleon_subnet['id']
network_id = chameleon_network_id

try:
    result = chi.network.remove_all_routes_from_router(router_id)
except Exception as e:
    print(f"detach_router error: {str(e)}")

In [ ]:
try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router error: {str(e)}")

In [ ]:
try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router error: {str(e)}")

In [ ]:
try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet error: {str(e)}")

In [ ]:
try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network error: {str(e)}")

In [ ]:
try:
    chi.lease.delete_lease(chameleon_lease['id'])
except Exception as e:
    print(f"delete_lease error: {str(e)}")

## Cleanup Fabric

In [ ]:
try:
    fabric_slice = fablib.get_slice(fabric_slice_name)
    fabric_slice.delete()
except Exception as e:
    print(f"Exception: {e}")